In [1]:
# pose sequence as a NLI premise and label as a hypothesis
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

In [2]:
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

In [7]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sequence = ['one day I will see the world', 'The food is good']
premise = sequence
hypothesis = ['This example is travel.', 'This example is food.']

# run through model pre-trained on MNLI
x = tokenizer(premise, hypothesis, return_tensors='pt',
                     truncation_strategy='only_first', padding=True, max_length=512)

/scratch/zhivar/anaconda3/envs/prototex/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2287: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [12]:
tokenizer.decode(x[0])

'<s>one day I will see the world</s></s>This example is travel.</s>'

In [15]:
logits = nli_model(x)[0]

# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (2) as the probability of the label being true 
entail_contradiction_logits = logits[:,[0,2]]
probs = entail_contradiction_logits.softmax(dim=1)
prob_label_is_true = probs[:,1]
